In [3]:
import numpy as np
import pdb

In [ ]:
def mape(X, predictions, y):
  num_timeslots = 43
  num_districts = 66
  if len(y.shape) == 1:
    y = np.asmatrix(y)
  if len(predictions.shape) == 1:
    predictions = np.asmatrix(predictions)
  Xy = np.concatenate(X, y.T, predictions.T)
  district_scores = np.array([])
  for key in np.unique(groups):
    district_scores[len(district_scores)] = (
      Xy[np.where()]
    )

def mape_scorer(estimator, X, y):
  estimator.fit(X)
  predictions = estimator.predict(X)
  return mape(X, predictions, y)

Testing MAPE

In [1]:
from sklearn.linear_model import LogisticRegression

est = LogisticRegression()
X = np.array([[1, 1], [1, 2], [2, 3], [2, 4]])
y = np.array([1, 2])

mape_scorer(est, X, y)

In [ ]:
features_list = ['']